# 前置設定

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
from torch.utils.data import DataLoader
from drive.MyDrive.iirlab.topic.LSAN.LSANori.trainer import *
from drive.MyDrive.iirlab.topic.LSAN.LSANori.transformer import *
from drive.MyDrive.iirlab.topic.LSAN.LSANori.dataset import *
from drive.MyDrive.iirlab.topic.LSAN.LSANori.LSAN import *
import pickle
import matplotlib.pyplot as plt

# 滾動式預測

## 跑模型

In [ ]:
with open('/content/drive/MyDrive/iirlab/topic/LSAN/LSANori/model_inputs/rolling_prediction/29_39/rolling_prediction.pickle','rb') as f:
  data = pickle.load(f)
  visit = data[0]
  label = data[1]
  value = data[3]
  dataset = Dataset(visit, label, value)

# time_interval = '12_with_gap_6/18_39'
# with open('/content/drive/MyDrive/iirlab/topic/trimmed_pickle/' + time_interval + '/final_test.pickle','rb') as f:
#   data = pickle.load(f)
#   visit = data[0]
#   label = data[1]
#   value = data[3]
#   dataset = Dataset(visit, label, value)

In [ ]:
output_path = '/content/drive/MyDrive/iirlab/topic/LSAN/LSANori/output/result.txt'
hidden = 256 
layers = 8
attn_heads = 8 
dropout = 0.1
batch_size = 32
epochs = 10
num_workers = 2
with_cuda = 1
lr = 0.0001
adam_weight_decay = 0.01 ###
adam_beta1 = 0.9
adam_beta2 = 0.999
model_path = '/content/drive/MyDrive/iirlab/topic/LSAN/LSANori/model_parameters/models.pth'

In [ ]:
data_loader = DataLoader(dataset, batch_size=batch_size, num_workers=num_workers, collate_fn=dataset.collate_fn,shuffle=False)
embedding_dim = hidden

test_model = LSAN(51, embedding_dim, transformer_hidden = hidden, attn_heads = attn_heads, transformer_dropout = dropout, transformer_layers = layers) 
test_model.load_state_dict(torch.load(model_path))
test_model.eval()

LSAN_test = LSAN_trainer(test_model, train_dataloader=data_loader, validate_dataloader=data_loader, test_dataloader=data_loader, with_cuda=with_cuda, lr=lr, output_dir=output_path)
LSAN_test.test(0)

tensor([0.9119, 0.8437, 0.7876, 0.7555, 0.8484, 0.9222, 0.9206, 0.9542, 0.9445,
        0.9281, 0.8969, 0.8913, 0.9562, 0.9102, 0.8597, 0.8667, 0.8383, 0.8379,
        0.8873, 0.8717, 0.7409, 0.9461, 0.9505, 0.9084, 0.7988, 0.8036, 0.8236,
        0.8522, 0.8385, 0.7933, 0.7565, 0.6894], device='cuda:0',
       grad_fn=<SqueezeBackward1>)
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       device='cuda:0')
tensor([0.8033, 0.7098, 0.7817, 0.7823, 0.9046, 0.9431, 0.9173, 0.9605, 0.9556,
        0.9539, 0.8019, 0.6667, 0.7036, 0.7642, 0.8412, 0.8374, 0.8358, 0.9616,
        0.9638, 0.9714, 0.9315, 0.9021, 0.8880, 0.8647, 0.7521, 0.8269, 0.7728,
        0.8773, 0.8513, 0.9750, 0.9708, 0.9784], device='cuda:0',
       grad_fn=<SqueezeBackward1>)
tensor([1, 1, 1, 1,

## 畫圖

In [ ]:
rolling = pd.read_csv("./rp_result_with_patient_code.csv").to_numpy()
selectedRolling = []
check = 0
for i in range(len(rolling)):
    check = 0
    for j in range(11):
        if rolling[i][j] < rolling[i][j+1]:
            check += 1
    if check >= 10:
        print(int(rolling[i][12]), (int(rolling[i][13])))